# Project 2: ISS Pass Predictor & Logger

---

**Project Overview**
--

This project creates a personalized International Space Station (ISS) visibility tracker that predicts when the station passes overhead and maintains a historical log of observations. We'll use Skyfield for satellite predictions while building a Pandas-based database to track and analyze pass patterns over time.

**Key Features:**
- Download ISS TLE (Two-Line Element) data from Celestrak
- Predict visible passes for the next 7 days using Skyfield
- Store predictions in structured DataFrames
- Log actual observations with weather and viewing success data
- Analyze patterns using Pandas groupby operations
- Visualize pass frequencies and observation success rates
- Persist data to CSV files for cross-session continuity

**Astronomy Context:**
The International Space Station orbits Earth every ~90 minutes. A "pass" occurs when it flies overhead and becomes visible - appearing as a bright moving star. TLE data contains orbital parameters that enable software to predict satellite positions. Altitude refers to height above the horizon in degrees (0° = horizon, 90° = directly overhead). Passes above ~30° altitude are easiest to observe.

---

### Methodology

**Pathway B: Pandas + Astropy + Skyfield**
- **Skyfield**: Satellite orbit calculations and pass predictions
- **Pandas**: Data manipulation, filtering, and analysis
- **Astropy**: Astronomical coordinate transformations and constellation identification
- **Matplotlib**: Data visualization and plotting

**Data Pipeline:**
1. Acquire current ISS TLE data from Celestrak
2. Calculate visible passes for user location
3. Structure predictions in DataFrame format
4. Log observations with metadata
5. Merge predictions with observations
6. Analyze patterns and generate visualizations
7. Persist data for future sessions

## 1. Project Setup, Dependencies, and Data Acquisition

This section establishes the foundation for our ISS tracking system by:
- Installing required dependencies
- Importing necessary libraries
- Configuring the observing location
- Acquiring current ISS orbital data
- Setting up data structures for predictions and observations

Let's begin by ensuring all required packages are available and properly imported.

### 1.1 Dependency Installation

We need several specialized libraries for this project:

- **pandas**: Data manipulation and analysis (core requirement)
- **numpy**: Numerical computing foundation
- **matplotlib**: Data visualization and plotting
- **skyfield**: Satellite orbit calculations and pass predictions
- **astropy**: Astronomical coordinate transformations
- **requests**: HTTP requests for downloading TLE data

**Note**: Skyfield will automatically download ephemeris data (~10MB) on first use, which is normal and expected.

In [ ]:
# Install necessary libraries if not already present
!pip install pandas numpy matplotlib skyfield astropy requests

# Standard imports
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt

# Skyfield imports
from skyfield.api import load, EarthSatellite, Topos

# Astropy imports
from astropy.coordinates import SkyCoord, EarthLocation
from astropy import units as u
from astropy.time import Time

### 1.3 Define Observing Location and Time Scale

We need to define our observing location and load the necessary astronomical data files. For this project, we'll use Columbus, Ohio as our default location, but you can easily change the coordinates to your own location.

**Key Components:**
- **Observer Location**: Latitude, longitude, and elevation
- **Skyfield Timescale**: For accurate time calculations
- **Ephemeris Data**: For precise satellite position calculations

In [ ]:
# Define observing location (Columbus, Ohio)
# You can change these coordinates to your own location
latitude = 39.9612    # North latitude in degrees
longitude = -82.9988  # West longitude in degrees (negative for western hemisphere)
elevation = 275        # Elevation above sea level in meters

print(f"Observing Location:")
print(f"Latitude: {latitude}° N")
print(f"Longitude: {longitude}° W")
print(f"Elevation: {elevation} m")

# Create Skyfield observer location
observer = Topos(latitude, longitude, elevation_m=elevation)
print(f"✓ Observer location created successfully")

# Load Skyfield timescale (this downloads ephemeris data on first run)
print("Loading Skyfield timescale...")
ts = load.timescale()
print("✓ Timescale loaded successfully")

# Load ephemeris data for precise calculations
print("Loading ephemeris data...")
eph = load('de421.bsp')  # This downloads ~10MB of data on first run
print("✓ Ephemeris data loaded successfully")

print("\n" + "="*50)
print("Location and time scale setup complete!")
print("="*50)

### 1.4 Acquire TLE Data for the ISS

TLE (Two-Line Element) data contains the orbital parameters needed to predict satellite positions. We'll download the current ISS TLE data directly from Celestrak, which provides real-time satellite tracking data.

**What is TLE Data?**
- Contains orbital elements for satellite position prediction
- Updated regularly to maintain accuracy
- Format: Two lines of text with specific orbital parameters
- ISS NORAD ID: 25544

In [ ]:
# Download ISS TLE data from Celestrak
print("Downloading ISS TLE data from Celestrak...")

# Celestrak URL for ISS TLE data
tle_url = "https://celestrak.org/NORAD/elements/stations.txt"

try:
    # Download the TLE data
    response = requests.get(tle_url)
    response.raise_for_status()  # Raise an exception for bad status codes
    
    # Parse the TLE data
    tle_lines = response.text.strip().split('\n')
    
    # Find ISS data (NORAD ID 25544)
    iss_tle = None
    for i, line in enumerate(tle_lines):
        if 'ISS' in line or '25544' in line:
            # ISS TLE data is typically 3 lines: name, line 1, line 2
            if i + 2 < len(tle_lines):
                iss_tle = tle_lines[i:i+3]
                break
    
    if iss_tle:
        print("✓ ISS TLE data downloaded successfully")
        print(f"ISS Name: {iss_tle[0].strip()}")
        print(f"TLE Line 1: {iss_tle[1].strip()}")
        print(f"TLE Line 2: {iss_tle[2].strip()}")
        
        # Create Skyfield satellite object
        satellite = EarthSatellite(iss_tle[1], iss_tle[2], iss_tle[0], ts)
        print("✓ ISS satellite object created successfully")
        
    else:
        print("✗ ISS TLE data not found in downloaded file")
        
except requests.exceptions.RequestException as e:
    print(f"✗ Error downloading TLE data: {e}")
    print("Using fallback TLE data...")
    
    # Fallback TLE data (may be outdated but allows testing)
    fallback_tle = [
        "ISS (ZARYA)",
        "1 25544U 98067A   24123.12345678  .00001234  00000-0  12345-4 0  9999",
        "2 25544  51.6400 123.4567 0001234 123.4567 234.5678 15.12345678901234"
    ]
    
    satellite = EarthSatellite(fallback_tle[1], fallback_tle[2], fallback_tle[0], ts)
    print("✓ Fallback satellite object created")

print("\n" + "="*50)
print("TLE data acquisition complete!")
print("="*50)

**Core Prediction Pipeline and Data Wrangling**
--

This section implements the Skyfield pass calculation and structures the output into the required Pandas DataFrame.

---

**Function to Calculate Visible Passes**

This function finds all visible passes of the ISS over the defined time window and location.

**Astropy Integration & Logging System**
--
This section introduces Astropy for coordinate calculation and sets up the observation log.

---

**Feature: Constellation Crossing (Astropy)**

We define a function to find which constellation the ISS is in at its maximum altitude, using Astropy's coordinate tools.

**Observation Log and Merge**

We define the structure of the observation log (observations.csv), create a dummy dataset, and merge it with the predictions.

**Analysis, Visualization, and Conclusion**
--
This section fulfills the analysis, visualization, and persistence requirements.

---

**Analysis: Pass Frequency per Day**

We analyze the frequency of passes over the next 7 days, which is a key requirement.

**Visualization: Pass Frequencies (Bar Chart)**

A bar chart visualizing the number of passes per day.

**Visualization: Observation Success Rate**

A pie chart showing the success rate of the recorded observations.

**Persistence: Save Predictions and Full Data**

Save the full dataset to a new CSV file to meet the persistence requirement.

**Conclusion and Individual Contribution**
--

---

**Discussion and Interpretation**

**Individual Contribution Statement**